# Chapter 3. EDA w/ DuckDB

- https://learning.oreilly.com/library/view/duckdb-up-and/9781098159689/ch04.html

## LOAD DATA

In [35]:
# load the sql extension
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [36]:
# # start an in memory duckdb database
# %sql duckdb://

In [37]:
# %sql?

In [38]:
# start an saved duckdb database
%sql duckdb:////Users/vamsi_mbmax/Library/CloudStorage/OneDrive-Personal/01_vam_PROJECTS/LEARNING/proj_Databases/dev_proj_Databases/pract-duckDB/data/MyDB.ddb

In [39]:
# # start an saved duckdb database
# %sql duckdb:///MyDB.ddb

## PERFORMING QUERIES

In [40]:
%sql SELECT * FROM "../data/airlines.csv"

Running query in 'duckdb:////Users/vamsi_mbmax/Library/CloudStorage/OneDrive-Personal/01_vam_PROJECTS/LEARNING/proj_Databases/dev_proj_Databases/pract-duckDB/data/MyDB.ddb'

IATA_CODE,AIRLINE
UA,United Air Lines Inc.
AA,American Airlines Inc.
US,US Airways Inc.
F9,Frontier Airlines Inc.
B6,JetBlue Airways
OO,Skywest Airlines Inc.
AS,Alaska Airlines Inc.
NK,Spirit Air Lines
WN,Southwest Airlines Co.
DL,Delta Air Lines Inc.


In [41]:
# alternatively use read_csv_auto

# %sql SELECT * FROM read_csv_auto("/Users/vamsi_mbmax/Library/CloudStorage/OneDrive-Personal/01_vam_PROJECTS/LEARNING/proj_Databases/dev_proj_Databases/pract-duckDB/data/airlines.csv", header=true)

In [42]:
result = %sql SELECT * FROM "../data/airlines.csv"
df = result.DataFrame()

df

Running query in 'duckdb:////Users/vamsi_mbmax/Library/CloudStorage/OneDrive-Personal/01_vam_PROJECTS/LEARNING/proj_Databases/dev_proj_Databases/pract-duckDB/data/MyDB.ddb'

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways
5,OO,Skywest Airlines Inc.
6,AS,Alaska Airlines Inc.
7,NK,Spirit Air Lines
8,WN,Southwest Airlines Co.
9,DL,Delta Air Lines Inc.


In [43]:
# display all rows by default
%config SqlMagic.displaylimit = None

displaylimit: Value None will be treated as 0 (no limit)

In [50]:
%%sql
CREATE TABLE airlines
AS
FROM "../data/airlines.csv"

Running query in 'duckdb:////Users/vamsi_mbmax/Library/CloudStorage/OneDrive-Personal/01_vam_PROJECTS/LEARNING/proj_Databases/dev_proj_Databases/pract-duckDB/data/MyDB.ddb'

Count


In [51]:
%sql SELECT * FROM airlines

Running query in 'duckdb:////Users/vamsi_mbmax/Library/CloudStorage/OneDrive-Personal/01_vam_PROJECTS/LEARNING/proj_Databases/dev_proj_Databases/pract-duckDB/data/MyDB.ddb'

IATA_CODE,AIRLINE
UA,United Air Lines Inc.
AA,American Airlines Inc.
US,US Airways Inc.
F9,Frontier Airlines Inc.
B6,JetBlue Airways
OO,Skywest Airlines Inc.
AS,Alaska Airlines Inc.
NK,Spirit Air Lines
WN,Southwest Airlines Co.
DL,Delta Air Lines Inc.


In [55]:
# check tables in the database
%sqlcmd tables

Name
airlines


In [56]:
# check columns of a table
%sqlcmd columns -t airlines

name,type,nullable,default,autoincrement,comment
IATA_CODE,VARCHAR,True,None,False,None
AIRLINE,VARCHAR,True,None,False,None


In [57]:
%sqlcmd profile -t airlines

,IATA_CODE,AIRLINE
count,14,14
unique,14,14
top,WN,Skywest Airlines Inc.
freq,1,1
min,nan,nan
max,nan,nan


In [60]:
%%sql --save state_count

SELECT
count(*) as Count, STATE
FROM "../data/airports.csv"
GROUP BY STATE
ORDER BY Count DESC
LIMIT 10;

Running query in 'duckdb:////Users/vamsi_mbmax/Library/CloudStorage/OneDrive-Personal/01_vam_PROJECTS/LEARNING/proj_Databases/dev_proj_Databases/pract-duckDB/data/MyDB.ddb'

Count,STATE
24,TX
22,CA
19,AK
17,FL
15,MI
14,NY
10,CO
8,ND
8,MT
8,MN


In [61]:
%sql SELECT * FROM state_count

Generating CTE with stored snippets: 'state_count'

Running query in 'duckdb:////Users/vamsi_mbmax/Library/CloudStorage/OneDrive-Personal/01_vam_PROJECTS/LEARNING/proj_Databases/dev_proj_Databases/pract-duckDB/data/MyDB.ddb'

Count,STATE
24,TX
22,CA
19,AK
17,FL
15,MI
14,NY
10,CO
8,PA
8,NC
8,WI


In [66]:
%%sql --save state_count --no-execute

SELECT
count(*) as Count, STATE
FROM "../data/airports.csv"
GROUP BY STATE
ORDER BY Count

Running query in 'duckdb:////Users/vamsi_mbmax/Library/CloudStorage/OneDrive-Personal/01_vam_PROJECTS/LEARNING/proj_Databases/dev_proj_Databases/pract-duckDB/data/MyDB.ddb'

Skipping execution...

In [67]:
%sql SELECT * FROM state_count

Generating CTE with stored snippets: 'state_count'

Running query in 'duckdb:////Users/vamsi_mbmax/Library/CloudStorage/OneDrive-Personal/01_vam_PROJECTS/LEARNING/proj_Databases/dev_proj_Databases/pract-duckDB/data/MyDB.ddb'

Count,STATE
1,CT
1,VT
1,RI
1,GU
1,DE
1,MD
1,WV
1,AS
1,NH
2,VI


## VISUALIZATION